# Trial of multilayer perceptron on landmark delta

In [131]:
# imports 
import h5py
from keras.utils.np_utils import to_categorical  
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf



## Opening Data

In [45]:
h5f = h5py.File('data_train_labels.h5','r')
train_y = h5f['data_train_labels'][:]
h5f.close()

In [46]:
h5f = h5py.File('data_train_landmarks.h5','r')
train_l = h5f['data_train_landmarks'][:]
h5f.close()

In [116]:
h5f = h5py.File('data_test_landmarks.h5','r')
test_l = h5f['data_test_landmarks'][:]
h5f.close()

In [117]:
delta_test = test_l[:,0,:,:]-test_l[:,-1,:,:]

In [76]:
 y = to_categorical(train_y)

In [1]:
# computing delta
delta = train_l[:,0,:,:]-train_l[:,-1,:,:]

NameError: name 'train_l' is not defined

In [51]:
BATCH_SIZE = 1
num_classes = 6 

## Making a tensorflow dataset

In [252]:

X_train, X_test, y_train, y_test = train_test_split(delta.reshape(540,136), y, test_size=0.1)

In [253]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)
)
train_dataset = train_dataset.batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)
)
test_dataset = test_dataset.batch(BATCH_SIZE)



In [184]:
image_batch, label_batch = next(iter(train_dataset))



In [326]:
# basic mlp model 
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(10,activation='relu'),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(6, activation='softmax')
        
    ]
)




In [86]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [329]:
# compiling model 
model.compile(loss='categorical_crossentropy', 
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                    metrics = ['accuracy'])

In [330]:
# fitting model 
model.fit(train_dataset,epochs=50, validation_data=test_dataset)

Epoch 1/50
486/486 [==============================] - 1s 2ms/step - loss: 2.0581 - accuracy: 0.1358 - val_loss: 1.7777 - val_accuracy: 0.2222
Epoch 2/50
486/486 [==============================] - 1s 2ms/step - loss: 1.7878 - accuracy: 0.1914 - val_loss: 1.7279 - val_accuracy: 0.2222
Epoch 3/50
486/486 [==============================] - 1s 1ms/step - loss: 1.7280 - accuracy: 0.2695 - val_loss: 1.6364 - val_accuracy: 0.3889
Epoch 4/50
486/486 [==============================] - 1s 1ms/step - loss: 1.5358 - accuracy: 0.3663 - val_loss: 1.5401 - val_accuracy: 0.3889
Epoch 5/50
486/486 [==============================] - 1s 1ms/step - loss: 1.5125 - accuracy: 0.3663 - val_loss: 1.5701 - val_accuracy: 0.3519
Epoch 6/50
486/486 [==============================] - 1s 1ms/step - loss: 1.4218 - accuracy: 0.4424 - val_loss: 1.4965 - val_accuracy: 0.3704
Epoch 7/50
486/486 [==============================] - 1s 2ms/step - loss: 1.3761 - accuracy: 0.4527 - val_loss: 1.4393 - val_accuracy: 0.4259
Epoch 

## Polar Coordinates

In [311]:
# trial with polar coordinates
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

In [315]:
lala = delta.copy()

for i in range(lala.shape[0]):
    for j in range(lala.shape[1]):
        lala[i,j] = cart2pol(lala[i,j,0], lala[i,j,1])

In [316]:
X_train, X_test, y_train, y_test = train_test_split(lala.reshape(540,136), y, test_size=0.1)

In [318]:

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)
)
train_dataset = train_dataset.batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)
)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [321]:
# training model 
model.fit(train_dataset,epochs=50, validation_data=test_dataset)

Epoch 1/50
486/486 [==============================] - 1s 2ms/step - loss: 1.1190 - accuracy: 0.5473 - val_loss: 1.2985 - val_accuracy: 0.3704
Epoch 2/50
486/486 [==============================] - 1s 1ms/step - loss: 1.0655 - accuracy: 0.5844 - val_loss: 1.2126 - val_accuracy: 0.4444
Epoch 3/50
486/486 [==============================] - 1s 2ms/step - loss: 0.9762 - accuracy: 0.6091 - val_loss: 1.0461 - val_accuracy: 0.5000
Epoch 4/50
486/486 [==============================] - 1s 2ms/step - loss: 0.9951 - accuracy: 0.5988 - val_loss: 1.0857 - val_accuracy: 0.5185
Epoch 5/50
486/486 [==============================] - 1s 2ms/step - loss: 0.9544 - accuracy: 0.6049 - val_loss: 1.1005 - val_accuracy: 0.4630
Epoch 6/50
486/486 [==============================] - 1s 2ms/step - loss: 0.9324 - accuracy: 0.6173 - val_loss: 1.1184 - val_accuracy: 0.4815
Epoch 7/50
486/486 [==============================] - 1s 2ms/step - loss: 0.9396 - accuracy: 0.6420 - val_loss: 1.0845 - val_accuracy: 0.5556
Epoch 

In [140]:
sample_submission = pd.read_csv("sampleSubmission.csv")

sample_submission["expression"] = pred.astype(int)

sample_submission.to_csv("sampleSubmission.csv", index=False)